# AWS-Athena Sample connector

In [ ]:
!pip install boto3
import boto3

In [ ]:
class AthenaConnector:
    def __init__(self, database, output_bucket):
        self.database = database
        self.output_bucket = output_bucket
        self.client = boto3.client('athena')
    
    def run_query(self, query):
        response = self.client.start_query_execution(
            QueryString=query,
            QueryExecutionContext={
                'Database': self.database
            },
            ResultConfiguration={
                'OutputLocation': f's3://{self.output_bucket}/'
            }
        )
        query_execution_id = response['QueryExecutionId']
        return query_execution_id
    
    def get_query_status(self, query_execution_id):
        response = self.client.get_query_execution(
            QueryExecutionId=query_execution_id
        )
        return response['QueryExecution']['Status']['State']
    
    def get_query_results(self, query_execution_id):
        response = self.client.get_query_results(
            QueryExecutionId=query_execution_id
        )
        result_set = response['ResultSet']
        columns = [column['Name'] for column in result_set['ResultSetMetadata']['ColumnInfo']]
        rows = [[data['VarCharValue'] for data in row['Data']] for row in result_set['Rows']]
        return columns, rows
    
    # Define other methods or functions for your Athena connector
    # ...
